## DATA COLLECTION

In [1]:
# importing default libraries
import requests
from bs4 import BeautifulSoup
import urllib
import re
import time
from urllib.request import Request, urlopen
import pandas as pd
import json

In [2]:
# importing  selenium libraries
from selenium import webdriver
from selenium.webdriver.common.by import By
import csv
# import chromedriver_autoinstaller to install chromedriver automatically
import chromedriver_autoinstaller
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver import Chrome
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

In [3]:
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.page_load_strategy = 'none'

chrome_path = ChromeDriverManager().install()
chrome_service = Service(chrome_path)

driver = Chrome(options, service = chrome_service)
driver.implicitly_wait(10)

[WDM] - Downloading: 100%|█████████████████████████████████████████████████████████| 6.30M/6.30M [00:08<00:00, 736kB/s]
C:\Users\USER\AppData\Local\Temp\ipykernel_7764\597241106.py:8: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = Chrome(options, service = chrome_service)


In [16]:
driver.get('https://ke.soccerway.com/national/kenya/premier-league/20222023/regular-season/r71051/')
time.sleep(10)

In [6]:
even_content = driver.find_elements(By.CSS_SELECTOR, "tr[class*='even  expanded    match border']")
print(len(even_content))

5


In [20]:
odd_content = driver.find_elements(By.CSS_SELECTOR, "tr[class*='odd  expanded    match border']")
print(len(content))

5


In [21]:
# get Home team, Result and Away team
gameweek = {"Home": [], "Result": [], "Away": []}
for contain in odd_content:
    try: 
        gameweek["Home"].append(contain.find_element(By.CSS_SELECTOR, "td[class*='team team-a ']>a").text)
    except: 
        continue
    try: 
        gameweek["Result"].append(contain.find_element(By.CSS_SELECTOR, "td[class*='score-time ']>a").text)
    except: 
        continue
    try: 
        gameweek["Away"].append(contain.find_element(By.CSS_SELECTOR, "td[class*='team team-b ']>a").text)
    except: 
        continue
for contain in even_content:
    try: 
        gameweek["Home"].append(contain.find_element(By.CSS_SELECTOR, "td[class*='team team-a ']>a").text)
    except: 
        continue
    try: 
        gameweek["Result"].append(contain.find_element(By.CSS_SELECTOR, "td[class*='score-time ']>a").text)
    except: 
        continue
    try: 
        gameweek["Away"].append(contain.find_element(By.CSS_SELECTOR, "td[class*='team team-b ']>a").text)
    except: 
        continue
# convert to  dataframe
gmwk_df = pd.DataFrame(gameweek)
gmwk_df

,Home,Result,Away
0,KCB,0 - 0,Sofapaka
1,Mathare United,0 - 1,KCB
2,Posta Rangers,1 - 2,Sofapaka
3,Tusker,0 - 0,Gor Mahia
4,Talanta,1 - 0,Homeboyz
5,Bidco United,3 - 0,Ulinzi Stars
6,Ulinzi Stars,1 - 1,Police
7,Nzoia Sugar,0 - 1,Kariobangi Sharks
8,Bidco United,3 - 3,Vihiga Bullets
9,Bandari,1 - 0,Wazito


In [17]:
# experiment on the previous button
prev_button = driver.find_element('xpath', '//a[@class="previous "]')
driver.execute_script('arguments[0].click();', prev_button)
print('Finallisima')

Finallisima


In [6]:
# get games from the 2022/2023 season
season = {}
for i in range(35):
    # find the elements
    #even  expanded    match border no-date-repetition                even  expanded    match border 
    even_content = driver.find_elements(By.CSS_SELECTOR, "tr[class*='even  expanded    match border']")
    odd_content = driver.find_elements(By.CSS_SELECTOR, "tr[class*='odd  expanded    match border']")
    # get Home team, Result and Away team
    contenting = [even_content, odd_content]
    gameweek = {"Home": [], "Result": [], "Away": []}
    for content in contenting:
        for contain in content:
            try:
                gameweek["Home"].append(contain.find_element(By.CSS_SELECTOR, "td[class*='team team-a ']>a").text)
            except: 
                continue
            try: 
                gameweek["Result"].append(contain.find_element(By.CSS_SELECTOR, "td[class*='score-time ']>a").text)
            except: 
                continue
            try: 
                gameweek["Away"].append(contain.find_element(By.CSS_SELECTOR, "td[class*='team team-b ']>a").text)
            except: 
                continue
    # save to dataframe
    gmwk_df = pd.DataFrame(gameweek)
    # save gameweek to dictionary
    game= 35 - i
    season[str(game)] = gmwk_df
    # experiment on the previous button
    try:
        prev_button = driver.find_element('xpath', '//a[@class="previous "]')
    except NoSuchElementException:
        try:
            prev_button = driver.find_element('xpath', '//a[@class="previous"]')
        except NoSuchElementException:
            break
    driver.execute_script('arguments[0].click();', prev_button)
    time.sleep(5)

In [7]:
# length of gameweeks
print(len(season))
# number of keys
print(season.keys())

31
dict_keys(['35', '34', '33', '32', '31', '30', '29', '28', '27', '26', '25', '24', '23', '22', '21', '20', '19', '18', '17', '16', '15', '14', '13', '12', '11', '10', '9', '8', '7', '6', '5'])


In [25]:
# split two columns by -
gameweek = season['34']
# add a game
# add row to an existing dataframe
gameweek[['Home Result', 'Away Result']] = gameweek.Result.str.split(" - ",expand=True)
# convert string to integer columns
gameweek['Home Result'] = gameweek['Home Result'].astype(int)
gameweek['Away Result'] = gameweek['Away Result'].astype(int)
gameweek.reset_index(drop=True, inplace=True)
gameweek

,Home,Result,Away,Home Result,Away Result
0,Police,3 - 1,Posta Rangers,3,1
1,Talanta,2 - 2,Nairobi City Stars,2,2
2,Bandari,1 - 0,Kariobangi Sharks,1,0
3,Mathare United,1 - 1,Ulinzi Stars,1,1
4,Leopards,1 - 2,Sofapaka,1,2
5,Wazito,2 - 1,Tusker,2,1
6,Tusker,1 - 1,Posta Rangers,1,1
7,Gor Mahia,2 - 3,Homeboyz,2,3
8,Nzoia Sugar,0 - 2,KCB,0,2
9,Police,4 - 1,Vihiga Bullets,4,1


In [ ]:
# add new row
# season['34']
#gameweek.loc[len(gameweek.index)] = ['Bidco United', '0 - 1', 'Wazito', 0, 1]
# delete row
# season['35']
#gameweek.drop(labels=0, axis=0, inplace=True)

In [8]:
# form the first table
teams = ['Bandari', 'Bidco United', 'Gor Mahia', 'Homeboyz', 'Kariobangi Sharks', 'KCB', 'Leopards', 'Mathare United', 
        'Nairobi City Stars', 'Nzoia Sugar', 'Police', 'Posta Rangers', 'Sofapaka', 'Talanta', 'Tusker', 'Ulinzi Stars', 
        'Vihiga Bullets', 'Wazito']

In [18]:
# sample teamTable
teamTable = pd.DataFrame()
teamTable['Teams'] = teams
teamTable['MP'] = 0
teamTable['Points'] = 0
teamTable['GS'] = 0
teamTable['GA'] = 0
teamTable['GD'] = 0

In [13]:
print(season['3'].shape[0])

7


In [9]:
# add MP, points, GS, GA, GD
def points_giver(table_match, table_rank):
    # look at each match
    for i in range(table_match.shape[0]):
        # add points
        if table_match['Home Result'][i] > table_match['Away Result'][i]:
            table_rank.loc[table_rank['Teams'] == table_match['Home'][i], ['Points']] = table_rank[table_rank['Teams'] == table_match['Home'][i]]['Points'] + 3
        elif table_match['Home Result'][i] < table_match['Away Result'][i]:
            table_rank.loc[table_rank['Teams'] == table_match['Away'][i], ['Points']] = table_rank[table_rank['Teams'] == table_match['Away'][i]]['Points'] + 3
        else:
            table_rank.loc[table_rank['Teams'] == table_match['Home'][i], ['Points']] = table_rank[table_rank['Teams'] == table_match['Home'][i]]['Points'] + 1
            table_rank.loc[table_rank['Teams'] == table_match['Away'][i], ['Points']] = table_rank[table_rank['Teams'] == table_match['Away'][i]]['Points'] + 1
        # MP, GS, GA, GD
        team_result = ['Home Result', 'Away Result']
        team_play = ['Home', 'Away']
        for j in range(len(team_play)):
            # MP
            table_rank.loc[table_rank['Teams'] == table_match[team_play[j]][i], ['MP']] = table_rank[table_rank['Teams'] == table_match[team_play[j]][i]]['MP'] + 1
            # GS
            table_rank.loc[table_rank['Teams'] == table_match[team_play[j]][i], ['GS']] = table_rank[table_rank['Teams'] == table_match[team_play[j]][i]]['GS'] + table_match[team_result[j]][i]
            # GA
            table_rank.loc[table_rank['Teams'] == table_match[team_play[j]][i], ['GA']] = table_rank[table_rank['Teams'] == table_match[team_play[j]][i]]['GA'] + table_match[team_result[j-1]][i]
    table_rank['GD'] = table_rank['GS'] - table_rank['GA']
    return table_rank

In [20]:
premier = points_giver(season['4'], teamTable)
premier

,Teams,MP,Points,GS,GA,GD
0,Bandari,0,0,0,0,0
1,Bidco United,1,3,2,0,2
2,Gor Mahia,1,3,1,0,1
3,Homeboyz,1,0,1,2,-1
4,Kariobangi Sharks,1,1,2,2,0
5,KCB,1,3,1,0,1
6,Leopards,1,3,2,0,2
7,Mathare United,0,0,0,0,0
8,Nairobi City Stars,1,0,0,2,-2
9,Nzoia Sugar,1,0,0,1,-1


In [21]:
premier.sort_values(['Points', 'GD', 'MP', 'GS'], ascending=[False, False, True, False], inplace=True)
premier

,Teams,MP,Points,GS,GA,GD
1,Bidco United,1,3,2,0,2
6,Leopards,1,3,2,0,2
14,Tusker,1,3,2,1,1
2,Gor Mahia,1,3,1,0,1
5,KCB,1,3,1,0,1
4,Kariobangi Sharks,1,1,2,2,0
13,Talanta,1,1,2,2,0
0,Bandari,0,0,0,0,0
7,Mathare United,0,0,0,0,0
11,Posta Rangers,0,0,0,0,0


In [36]:
df_pts = pd.DataFrame()
df_gd = pd.DataFrame()
df_pts['Teams'] = teams
df_gd['Teams'] = teams
df_pts['1'] = 0
df_gd['1'] = 0
for tm in teams:
    # points and goal difference
    points = premier[premier['Teams'] == tm]['Points']
    gd = premier[premier['Teams'] == tm]['GD']
    df_pts.loc[df_pts['Teams'] == tm, ['1']] = int(points)
    df_gd.loc[df_gd['Teams'] == tm, ['1']] = int(gd)

NameError: name 'premier' is not defined

In [25]:
df_gd

,Teams,1
0,Bandari,0
1,Bidco United,2
2,Gor Mahia,1
3,Homeboyz,-1
4,Kariobangi Sharks,0
5,KCB,1
6,Leopards,2
7,Mathare United,0
8,Nairobi City Stars,-2
9,Nzoia Sugar,-1


In [19]:
# position dataframe
finalissima_df = pd.DataFrame()
finalissima_df['Teams'] = teams
finalissima_df.head(10)

,Teams
0,Bandari
1,Bidco United
2,Gor Mahia
3,Homeboyz
4,Kariobangi Sharks
5,KCB
6,Leopards
7,Mathare United
8,Nairobi City Stars
9,Nzoia Sugar


In [10]:
#position of each team after gameweek
def acquire_pos(table_rank, table_pos, gw):
    # sort values in table
    table_rank.sort_values(['Points', 'GD', 'MP', 'GS'], ascending=[False, False, True, False], inplace=True)
    table_rank.reset_index(drop=True, inplace=True)
    i = table_rank.index
    output = []
    for i in range(table_pos.shape[0]):
        timu = table_rank[table_rank['Teams'] == table_pos['Teams'][i]].index.tolist()
        output.append(int(timu[0]) + 1)
    table_pos[str(gw)] = output
    return table_pos, table_rank

In [37]:
# points and goal difference in gameweek
def pts_gd(df_giver, timu):
    # output dictionary
    output_pts = {}
    output_gd = {}
    for tm in timu:
        # points and goal difference
        points = df_giver[df_giver['Teams'] == tm]['Points']
        gd = df_giver[df_giver['Teams'] == tm]['GD']
        # add values to dictionary
        output_pts[tm] = int(points)
        output_gd[tm] = int(gd)
    return output_pts, output_gd

In [30]:
print(acquire_pos(premier, finalissima_df, 1))

NameError: name 'premier' is not defined

In [38]:
def entire_season(dict_df, team_list):
    # form the first table
    teamTable = pd.DataFrame()
    # CREATE DEFAULT TABLE
    teamTable['Teams'] = team_list
    teamTable['MP'] = 0
    teamTable['Points'] = 0
    teamTable['GS'] = 0
    teamTable['GA'] = 0
    teamTable['GD'] = 0
    # positon dataframe
    finalissima_df = pd.DataFrame()
    finalissima_df['Teams'] = team_list
    # Points and GD dataframe
    points_df = pd.DataFrame()
    gd_df = pd.DataFrame()
    points_df['Teams'] = team_list
    gd_df['Teams'] = team_list
    # gameweek
    gmwk = 1
    ## loop through gameweeks
    for i in range(5,36):
        # split two columns by -
        gameweek = dict_df[str(i)]
        gameweek[['Home Result', 'Away Result']] = gameweek.Result.str.split(" - ",expand=True)
        # convert string to integer columns
        gameweek['Home Result'] = gameweek['Home Result'].astype(int)
        gameweek['Away Result'] = gameweek['Away Result'].astype(int)
        # add points, evaluate games
        premier = points_giver(gameweek, teamTable)
        # position of each team after gameweek
        changes = acquire_pos(premier, finalissima_df, gmwk)
        # edit
        teamTable = changes[1]
        finalissima_df = changes[0]
        # POINTS and GD DATAFRAME
        peaks = pts_gd(teamTable, team_list)
        points_df[str(gmwk)] = points_df['Teams'].map(peaks[0])
        gd_df[str(gmwk)] = gd_df['Teams'].map(peaks[1])
        gmwk += 1
    return finalissima_df, teamTable, points_df, gd_df

In [39]:
ladders = entire_season(season, teams)

In [34]:
ladders[1]

,Teams,MP,Points,GS,GA,GD
0,Gor Mahia,34,70,53,22,31
1,Tusker,34,69,45,23,22
2,Police,34,64,57,22,35
3,Nzoia Sugar,34,64,43,23,20
4,KCB,34,64,37,19,18
5,Bandari,34,60,44,27,17
6,Leopards,34,51,32,25,7
7,Homeboyz,34,49,35,27,8
8,Ulinzi Stars,34,49,35,34,1
9,Sofapaka,34,42,32,37,-5


In [41]:
ladders[3]

,Teams,1,2,3,4,5,6,7,8,9,...,22,23,24,25,26,27,28,29,30,31
0,Bandari,0,1,0,-1,-1,-2,-2,-1,-3,...,6,7,4,4,9,12,12,15,16,17
1,Bidco United,2,1,0,-1,-2,1,1,2,3,...,-3,-4,-1,-3,0,0,3,2,1,1
2,Gor Mahia,1,2,2,3,2,4,5,5,5,...,25,25,26,28,28,27,27,29,28,31
3,Homeboyz,-1,1,3,3,3,3,2,2,3,...,5,7,9,10,10,9,7,7,8,8
4,Kariobangi Sharks,0,1,1,1,1,0,0,-1,3,...,3,2,3,3,3,1,2,2,1,0
5,KCB,1,0,0,1,1,1,2,4,4,...,12,14,14,13,13,14,15,16,18,18
6,Leopards,2,2,1,0,1,-2,-3,-2,0,...,10,8,8,8,8,9,9,7,6,7
7,Mathare United,0,0,0,0,-3,-3,-8,-9,-10,...,-21,-26,-27,-28,-31,-35,-36,-37,-37,-36
8,Nairobi City Stars,-2,-5,-6,-7,-8,-10,-9,-10,-9,...,-10,-9,-9,-9,-9,-9,-9,-9,-9,-12
9,Nzoia Sugar,-1,0,1,2,4,4,4,8,6,...,15,19,19,20,20,20,21,22,20,20


In [42]:
# save to csv
ladders[0].to_csv('KPL Ranks.csv', index=False)
ladders[1].to_csv('KPL Table.csv', index=False)
ladders[2].to_csv('KPL Points.csv', index=False)
ladders[3].to_csv('KPL GD.csv', index=False)